In [25]:
import pandas as pd
import numpy as np
from pathlib import Path

# original
LAT_MIN, LAT_MAX = 51.4, 51.58          
LON_MIN, LON_MAX = -0.25,  0.07        

mean_lat = (LAT_MIN + LAT_MAX) / 2.0
km_per_deg_lat = 111.32
km_per_deg_lon = 111.32 * np.cos(np.deg2rad(mean_lat))

height_km = (LAT_MAX - LAT_MIN) * km_per_deg_lat
width_km = (LON_MAX - LON_MIN) * km_per_deg_lon
area_km2 = height_km * width_km

print(f"Height: {height_km:.2f} km")
print(f"Width : {width_km:.2f} km")
print(f"Total area: {area_km2:.2f} km²")

CELL_DEG = 0.0075 
CELL_FILE_ADDITION = "original_0075_v2"

Height: 20.04 km
Width : 22.18 km
Total area: 444.44 km²


Creating Bounds & Obtaining Grid Size

In [26]:
PROJECT_ROOT = Path.cwd().resolve().parents[0]     
COMBINED_DIR   = PROJECT_ROOT / "combined_path"
OG_DIR   = COMBINED_DIR / "new_test" / "original"
TRIPS_PATH = COMBINED_DIR / "trips_all_drivers.parquet" 
    
df = pd.read_parquet(TRIPS_PATH)
print(len(df))
inside = (
      (df["begin_lat"].between(LAT_MIN, LAT_MAX))
  &   (df["end_lat"]  .between(LAT_MIN, LAT_MAX))
  &   (df["begin_lng"].between(LON_MIN, LON_MAX))
  &   (df["end_lng"]  .between(LON_MIN, LON_MAX))
)
df = df.loc[inside].copy()
print("Trips inside box:", len(df))


701156
Trips inside box: 148693


In [ ]:
# row 0 = north-most band, col 0 = west-most column
df["row"] = ((df["begin_lat"] - LAT_MIN) // CELL_DEG).astype(int)
df["col"] = ((df["begin_lng"] - LON_MIN) // CELL_DEG).astype(int)


n_rows = int(round((LAT_MAX - LAT_MIN) / CELL_DEG))
n_cols = int(round((LON_MAX - LON_MIN) / CELL_DEG))
print("num rows", n_rows, "num cols", n_cols)  

max_row = n_rows - 1 
max_col = n_cols -1

df["row"] = df["row"].clip(0, max_row)
df["col"] = df["col"].clip(0, max_col)



df["hour"] = df["begin_timestamp_local"].dt.hour

# aggregate → count of trips per (hour, row, col)
cube = (
    df.groupby(["hour", "row", "col"], observed=True)
      .size()
      .rename("n_trips")
      .reset_index()
)

mean_lat = (LAT_MIN + LAT_MAX) / 2.0
km_per_deg_lat = 111.32
km_per_deg_lon = 111.32 * np.cos(np.deg2rad(mean_lat))

cell_h_km = CELL_DEG * km_per_deg_lat            # north-south
cell_w_km = CELL_DEG * km_per_deg_lon            # east-west
cell_area_km2 = cell_h_km * cell_w_km

print(f"Grid: {n_rows} rows × {n_cols} cols")
print(f"Cell size = {cell_h_km:.2f} km (N–S) × {cell_w_km:.2f} km (E–W) at {mean_lat:.3f}°N"
      f" area = {cell_area_km2:.2f} km²")


num rows 24 num cols 43
Grid: 24 rows × 43 cols
Cell size = 0.83 km (N–S) × 0.52 km (E–W) at 51.490°N area = 0.43 km²


Hourly Info of Ride Coverage (Optional Images)

In [28]:
non_zero_info = []
for hour in range(24):
    hr = cube.loc[cube["hour"] == hour]

    # heat array uses NW origin: row 0 is north-most, col 0 is west-most
    heat = np.zeros((n_rows, n_cols), dtype=int)

    # clip (just in case) and accumulate
    r = hr["row"].clip(0, n_rows - 1).to_numpy()
    c = hr["col"].clip(0, n_cols - 1).to_numpy()
    v = hr["n_trips"].to_numpy()
    np.add.at(heat, (r, c), v)

    total_cells = n_rows * n_cols
    nonzero = (heat > 0).sum()
    zero = total_cells - nonzero
    non_zero_info.append((hour, nonzero, zero, total_cells))
    print(f"Hour {hour:02d}: zero-ride cells = {zero:,} / {total_cells:,} (nonzero = {nonzero:,})")

    # plt.figure(figsize=(10, 4))
    # plt.imshow(
    #     heat,
    #     origin="upper",  # put row 0 (north) at the top
    #     extent=[LON_MIN, LON_MAX, LAT_MIN, LAT_MAX],  # x: lon (W→E), y: lat (S→N)
    #     interpolation="nearest",
    #     aspect="auto",
    # )
    # plt.colorbar(label=f"Pick-ups at {hour:02d}:00–{hour:02d}:59")
    # plt.title(f"Central London pick-up density (cell = {CELL_DEG:.02f}°) — {hour:02d}:00 h")
    # plt.xlabel("Longitude")
    # plt.ylabel("Latitude")
    # plt.tight_layout()
    # plt.show()

Hour 00: zero-ride cells = 219 / 1,032 (nonzero = 813)
Hour 01: zero-ride cells = 312 / 1,032 (nonzero = 720)
Hour 02: zero-ride cells = 394 / 1,032 (nonzero = 638)
Hour 03: zero-ride cells = 450 / 1,032 (nonzero = 582)
Hour 04: zero-ride cells = 512 / 1,032 (nonzero = 520)
Hour 05: zero-ride cells = 513 / 1,032 (nonzero = 519)
Hour 06: zero-ride cells = 461 / 1,032 (nonzero = 571)
Hour 07: zero-ride cells = 298 / 1,032 (nonzero = 734)
Hour 08: zero-ride cells = 231 / 1,032 (nonzero = 801)
Hour 09: zero-ride cells = 205 / 1,032 (nonzero = 827)
Hour 10: zero-ride cells = 235 / 1,032 (nonzero = 797)
Hour 11: zero-ride cells = 232 / 1,032 (nonzero = 800)
Hour 12: zero-ride cells = 226 / 1,032 (nonzero = 806)
Hour 13: zero-ride cells = 208 / 1,032 (nonzero = 824)
Hour 14: zero-ride cells = 210 / 1,032 (nonzero = 822)
Hour 15: zero-ride cells = 168 / 1,032 (nonzero = 864)
Hour 16: zero-ride cells = 150 / 1,032 (nonzero = 882)
Hour 17: zero-ride cells = 132 / 1,032 (nonzero = 900)
Hour 18: z

In [29]:
df_info = pd.DataFrame(non_zero_info, columns=[
    "Hour", "Nonzero Cells", "Zero Cells", "Total Cells"
])


df_info["% Nonzero Cells"] = (df_info["Nonzero Cells"] / df_info["Total Cells"]) * 100
df_info["% Nonzero Cells"] = df_info["% Nonzero Cells"].round(2)

from IPython.display import display
display(df_info)

,Hour,Nonzero Cells,Zero Cells,Total Cells,% Nonzero Cells
0,0,813,219,1032,78.78
1,1,720,312,1032,69.77
2,2,638,394,1032,61.82
3,3,582,450,1032,56.40
4,4,520,512,1032,50.39
5,5,519,513,1032,50.29
6,6,571,461,1032,55.33
7,7,734,298,1032,71.12
8,8,801,231,1032,77.62
9,9,827,205,1032,80.14


Generation of Hourly Probabilities

In [30]:
total_cube = cube["n_trips"].sum()
print("Total trips in cube:", total_cube)


print("Total trips in df  :", len(df))

hourly_totals = (
    cube.groupby("hour", observed=True)["n_trips"]
        .sum()
        .sort_index()         
)
rides_per_epoch = (hourly_totals / 60).astype(float)  

print(hourly_totals)
print(rides_per_epoch)

hourly_prob = np.zeros((24, n_rows, n_cols), dtype=float)


for h in range(24):
    sub = cube.loc[cube["hour"] == h]
    if sub.empty:
        continue

    per_cell = (
        sub.groupby(["row", "col"], observed=True)["n_trips"]
           .sum()
           .rename("count")
           .reset_index()
    )
    hour_total = per_cell["count"].sum()
    if hour_total == 0:
        continue

    per_cell["prob"] = per_cell["count"] / hour_total

    rows = per_cell["row"].to_numpy(dtype=int)
    cols = per_cell["col"].to_numpy(dtype=int)
    probs = per_cell["prob"].to_numpy(dtype=float)

    hourly_prob[h, rows, cols] = probs
    
    slice_sum = hourly_prob[h].sum()
    if slice_sum:                               # guard against divide-by-zero
        hourly_prob[h] /= slice_sum             # renormalise
    # optional sanity-check:
        assert np.isclose(hourly_prob[h].sum(), 1.0), f"Hour {h} does not sum to 1"


for h in range(24):
    print(f"hour {h}: total={hourly_prob[h].sum():.6f}")

print(hourly_prob[2][1][22])


OUT_FILE = OG_DIR / f"hourly_origin_prob_grid_{CELL_FILE_ADDITION}_no_epsilon.npy"
np.save(OUT_FILE, hourly_prob)  

Total trips in cube: 135452
Total trips in df  : 148693
hour
0.0     6210
1.0     4840
2.0     3509
3.0     2527
4.0     1591
5.0     1371
6.0     1737
7.0     2926
8.0     4018
9.0     4468
10.0    4534
11.0    4783
12.0    5039
13.0    5594
14.0    5923
15.0    6722
16.0    7477
17.0    8568
18.0    9855
19.0    9781
20.0    8644
21.0    8639
22.0    8842
23.0    7854
Name: n_trips, dtype: int64
hour
0.0     103.500000
1.0      80.666667
2.0      58.483333
3.0      42.116667
4.0      26.516667
5.0      22.850000
6.0      28.950000
7.0      48.766667
8.0      66.966667
9.0      74.466667
10.0     75.566667
11.0     79.716667
12.0     83.983333
13.0     93.233333
14.0     98.716667
15.0    112.033333
16.0    124.616667
17.0    142.800000
18.0    164.250000
19.0    163.016667
20.0    144.066667
21.0    143.983333
22.0    147.366667
23.0    130.900000
Name: n_trips, dtype: float64
hour 0: total=1.000000
hour 1: total=1.000000
hour 2: total=1.000000
hour 3: total=1.000000
hour 4: total=1.

In [31]:
# Check what the data actually uses
print("row min/max:", cube["row"].min(), cube["row"].max())
print("col min/max:", cube["col"].min(), cube["col"].max())

n_rows_used = cube["row"].max() + 1
n_cols_used = cube["col"].max() + 1
print("n_rows in file:", n_rows, "vs. used:", n_rows_used)
print("n_cols in file:", n_cols, "vs. used:", n_cols_used)

# Which rows/cols are completely empty across all hours?
arr = np.load(OUT_FILE)  # hourly_prob
empty_rows = np.where(arr.sum(axis=(0,2)) == 0)[0]
empty_cols = np.where(arr.sum(axis=(0,1)) == 0)[0]
print("empty rows:", empty_rows)
print("empty cols:", empty_cols)

import numpy as np

print("data lat min/max:", df["begin_lat"].min(), df["begin_lat"].max())
print("grid LAT_MIN/MAX:", LAT_MIN, LAT_MAX)
print("data lon min/max:", df["begin_lng"].min(), df["begin_lng"].max())
print("grid LON_MIN/MAX:", LON_MIN, LON_MAX)

lat_edges = np.linspace(LAT_MIN, LAT_MAX, n_rows + 1)
lon_edges = np.linspace(LON_MIN, LON_MAX, n_cols + 1)

print("last lat bin:", (lat_edges[-2], lat_edges[-1]))
print("last lon bin:", (lon_edges[-2], lon_edges[-1]))

# counts using half-open [lo, hi) — typical when you do floor/digitize(right=False)
in_last_row_open   = ((df["begin_lat"] >= lat_edges[-2]) & (df["begin_lat"] <  lat_edges[-1])).sum()
in_last_col_open   = ((df["begin_lng"] >= lon_edges[-2]) & (df["begin_lng"] <  lon_edges[-1])).sum()

# counts if you instead included the top edge (lo, hi]
in_last_row_closed = ((df["begin_lat"] >  lat_edges[-2]) & (df["begin_lat"] <= lat_edges[-1])).sum()
in_last_col_closed = ((df["begin_lng"] >  lon_edges[-2]) & (df["begin_lng"] <= lon_edges[-1])).sum()

print("last row [lo,hi):", in_last_row_open, "   last row (lo,hi]:", in_last_row_closed)
print("last col [lo,hi):", in_last_col_open, "   last col (lo,hi]:", in_last_col_closed)


H, yedges, xedges = np.histogram2d(
    df["begin_lat"], df["begin_lng"],
    bins=(lat_edges, lon_edges)
)
print("H shape:", H.shape)  # expect (n_rows, n_cols)
print("empty rows:", np.where(H.sum(axis=1) == 0)[0])
print("empty cols:", np.where(H.sum(axis=0) == 0)[0])





row min/max: 0 23
col min/max: 0 42
n_rows in file: 24 vs. used: 24
n_cols in file: 43 vs. used: 43


empty rows: []
empty cols: []
data lat min/max: 51.40000534057617 51.57999420166016
grid LAT_MIN/MAX: 51.4 51.58
data lon min/max: -0.2499867677688598 0.0699918121099472
grid LON_MIN/MAX: -0.25 0.07
last lat bin: (np.float64(51.5725), np.float64(51.58))
last lon bin: (np.float64(0.06255813953488376), np.float64(0.07))
last row [lo,hi): 1418    last row (lo,hi]: 1418
last col [lo,hi): 402    last col (lo,hi]: 402
H shape: (24, 43)
empty rows: []
empty cols: []


Adding Rows to Table

In [32]:
OUT_PARQUET     = OG_DIR / f"trips_with_all_drivers_added_cells_{CELL_FILE_ADDITION}.parquet"
OUT_CSV     = OG_DIR / f"trips_with_all_drivers_added_cells_{CELL_FILE_ADDITION}.csv"

OUT_PARQUET.parent.mkdir(parents=True, exist_ok=True)


df["raw_origin_row"] = (df["begin_lat"] - LAT_MIN) // CELL_DEG
df["raw_origin_col"] = (df["begin_lng"] - LON_MIN) // CELL_DEG
df["raw_dest_row"]   = (df["end_lat"]   - LAT_MIN) // CELL_DEG
df["raw_dest_col"]   = (df["end_lng"]   - LON_MIN) // CELL_DEG


def clean_index(raw_series: pd.Series, max_idx: int) -> pd.Series:
    cond_bad = raw_series.isna() | (raw_series < 0) | (raw_series >= max_idx)
    out = raw_series.copy()
    out[cond_bad] = np.nan
    return out.fillna(-1).astype(int)



df["origin_row"] = clean_index(df["raw_origin_row"], n_rows)
df["origin_col"] = clean_index(df["raw_origin_col"], n_cols)
df["dest_row"]   = clean_index(df["raw_dest_row"],   n_rows)
df["dest_col"]   = clean_index(df["raw_dest_col"],   n_cols)


df = df.drop(columns=["raw_origin_row", "raw_origin_col",
                      "raw_dest_row",   "raw_dest_col"])


df["origin_row"] = np.where(df["origin_row"].between(0, max_row), df["origin_row"], -1)
df["origin_col"] = np.where(df["origin_col"].between(0, max_col), df["origin_col"], -1)
df["dest_row"]   = np.where(df["dest_row"].between(0, max_row), df["dest_row"], -1)
df["dest_col"]   = np.where(df["dest_col"].between(0,max_col), df["dest_col"], -1)


print(f"Writing {len(df)} rows ")
df.to_parquet(OUT_PARQUET, compression="zstd")
df.to_csv(OUT_CSV, index=False, float_format="%.6f")


print("Done. Now you can load the file with origin_row/origin_col/dest_row/dest_col already present.")


Writing 148693 rows 
Done. Now you can load the file with origin_row/origin_col/dest_row/dest_col already present.


Creating Final Table

In [33]:
# Output—slim version with local times and all required columns
FINAL_PARQUET = OG_DIR / f"trips_with_price_duration{CELL_FILE_ADDITION}_km.parquet"
FINAL_CSV     = OG_DIR / f"trips_with_price_duration{CELL_FILE_ADDITION}_km.csv"
FINAL_PARQUET.parent.mkdir(parents=True, exist_ok=True)


UTC_BEGIN_COL  = "begintrip_timestamp_utc"
UTC_END_COL    = "dropoff_timestamp_utc"
PRICE_COL      = "pay_after_uber_cut"
DUR_MS_COL     = "duration_ms"


R = 6_371.0 

lat1 = np.radians(df["begin_lat"].astype(float))
lon1 = np.radians(df["begin_lng"].astype(float))
lat2 = np.radians(df["end_lat"].astype(float))
lon2 = np.radians(df["end_lng"].astype(float))

dlat = lat2 - lat1
dlon = lon2 - lon1

a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
df["haversine_km"] = R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

print("Added 'haversine_km' column.")

before = len(df)
df = df[df[UTC_BEGIN_COL].notna()].copy()
after = len(df)
print(f"Dropped {before - after} rows lacking '{UTC_BEGIN_COL}'.")


df[UTC_BEGIN_COL] = pd.to_datetime(df[UTC_BEGIN_COL], utc=True, errors="coerce")
df[UTC_END_COL]   = pd.to_datetime(df[UTC_END_COL],   utc=True, errors="coerce")


df["begintrip_timestamp_london"] = df[UTC_BEGIN_COL].dt.tz_convert("Europe/London")
df["dropoff_timestamp_london"]   = df[UTC_END_COL]  .dt.tz_convert("Europe/London")


df["hour"] = df["begintrip_timestamp_london"].dt.hour

if DUR_MS_COL in df.columns:
    df["duration_sec"] = df[DUR_MS_COL].astype(float) / 1000.0
    print("Converted 'duration_ms' to 'duration_sec'.")
else:
    raise KeyError(f"Column '{DUR_MS_COL}' not found; cannot compute duration.")


if PRICE_COL not in df.columns:
    raise KeyError(f"Column '{PRICE_COL}' not found; cannot keep price.")
df[PRICE_COL] = pd.to_numeric(df[PRICE_COL], errors="coerce")


required = [
    "origin_row", "origin_col", "dest_row", "dest_col",
    "begin_lat", "begin_lng", "end_lat", "end_lng",
    "begintrip_timestamp_london", "dropoff_timestamp_london", "hour",
    "duration_sec", PRICE_COL, "driver_id_offline_online", "trip_distance_miles"
]

df["trip_distance_km"] = df["trip_distance_miles"].astype(float) * 1.60934


before2 = len(df)
df = df.dropna(subset=required)
after2 = len(df)
print(f"Dropped {before2 - after2} rows missing any of {required}.")


keep_cols = [
    "origin_row", "origin_col", "dest_row", "dest_col",
    "begin_lat", "begin_lng", "end_lat", "end_lng",
    "haversine_km",    
    "begintrip_timestamp_london", "dropoff_timestamp_london", "hour",
    "duration_sec", PRICE_COL, "driver_id_offline_online", "trip_distance_miles", "trip_distance_km"
]

df_slim = df[keep_cols]
df_slim.to_parquet(FINAL_PARQUET, compression="zstd")
print("✅ Wrote Parquet →", FINAL_PARQUET)

df_slim.to_csv(FINAL_CSV, index=False, float_format="%.6f")
print("✅ Wrote CSV →", FINAL_CSV)


Added 'haversine_km' column.
Dropped 66 rows lacking 'begintrip_timestamp_utc'.
Converted 'duration_ms' to 'duration_sec'.
Dropped 1129 rows missing any of ['origin_row', 'origin_col', 'dest_row', 'dest_col', 'begin_lat', 'begin_lng', 'end_lat', 'end_lng', 'begintrip_timestamp_london', 'dropoff_timestamp_london', 'hour', 'duration_sec', 'pay_after_uber_cut', 'driver_id_offline_online', 'trip_distance_miles'].
✅ Wrote Parquet → C:\Users\aless\OneDrive - Nexus365\Thesis\driver_data\combined_path\new_test\original\trips_with_price_durationoriginal_0075_v2_km.parquet
✅ Wrote CSV → C:\Users\aless\OneDrive - Nexus365\Thesis\driver_data\combined_path\new_test\original\trips_with_price_durationoriginal_0075_v2_km.csv
